# Information-theoretic Regularization in Neural Networks

### 07/11/2018, Heinke Hihn, heinke.hihn@uni-ulm.de, Ulm University

## Bounded Rationality

In Maximum Expected Utility Theory an agent tries to maximize its expecte utility by following some policy $p(a|w)$:

\begin{equation}
\max_{p(a|w)} \sum_w \rho(w) \sum_{a}{p(a|w)U(w, a)},
\end{equation}

where $a$ is an action from the action space $A$ and $w$ is a world state from the world state space $W$, and $\U(w,a)$ is a utility function. We assume that the world states are distributed according to a known and fixed distribution $\rho(w)$ and that the world sates $w$ are finite and discrete. In the case of a single world state or world state distribution $p(w)=\delta(w-w_0)$, the decision-making problem simplifies into a single function optimization problem $a^* = \argmax_a \U(a)$. In many cases, solving such optimization problems may require an exhaustive search, where simple enumeration is extremely expensive.

A bounded rational decision maker (Ortega and Braun, 2013) tackles the above decision-making problem by settling on a good enough solution. Finding a bounded optimal policy requires to maximize the utility function while simultaneously remaining within some given constraints. The resulting policy is a conditional probability distribution $p(a|w)$, which essentially consists of choosing an action $a$ given a particular world state $w$. The constraints of limited information-processing resources can be formalized by setting an upper bound on the DKL (say B bits) that the decision-maker is maximally allowed to spend to transform its prior strategy into a posterior strategy through deliberation. This results in the following constrained optimization problem (Genewein et al., 2015):

\begin{equation}
\max_{p(a|w)} \sum_w \rho(w) \sum_{a}{p(a|w)U(w, a)}, \text{ s.t. } \mathrm{DKL}(p(a|w)||p(a)) \leq \text{B}.
\end{equation}

This constrained optimization problem can be formulated as an unconstrained problem (Ortega and Braun, 2013):
\begin{equation}
\max_{p(a|w)} \left( \sum_w \rho(w) \sum_{a}{p(a|w)U(w, a) - \frac{1}{\beta}\DKL(p(a|w)||p(a))} \right),
\label{eq:bounded}
\end{equation}

where the inverse temperature $\beta \in \mathbb{R}^+$ is a Lagrange multiplier that influences the trade off between expected utility gain and information cost. For $\beta \rightarrow \infty$ the agent behaves perfectly rational and for $\beta \rightarrow 0$ the agent can only act according to the prior policy. The optimal prior policy in this case is given by the marginal $p(a) = \sum_{w \in W}{\rho(w)  p(a|w)}$ (Ortega and Braun, 2013), in which case the Kullback-Leibler divergence becomes equal to the mutual information, i.e. $\mathrm{DKL}(p(a|w)||p(a))=I(W;A)$. The solution to the above optimization problem can be found by iterating the following set of self-consistent equations (Ortega and Braun, 2013):

\begin{cases}
\begin{array}{rcl}
p(a|w) &=& \frac{1}{Z(w)}p(a) \exp(\beta_1 \U(w,a)) \\
p(a) &=& \sum_w \rho(w) p(a|w), \\
\end{array}
\end{cases}

where $Z(w) = \sum_a p(a) \exp(\beta_1 \U(w,a)) $ is normalization factor. Computing such a normalization factor is usually computationally expensive as it involves summing over spaces with high cardinality. We avoid this by Monte Carlo approximation.

### An Online Update Principle
Let $\theta$ be the parameters of a artifial neural network (ANN), which we wish to find through online updates. The corresponding objective function is then

\begin{equation}
\max_{p_\theta(a|w)} \left( \sum_{w,a}\rho(w) p_\theta(a\vert w)\cdot \underbrace{\left[\U(w,a) - \frac{1}{\beta} \log \frac{p_\theta(a\vert w)}{p_\theta(a)} \right]}_{=~j(w,a)} \right),
\end{equation}
where $p_\theta(a) = \sum_w \rho(w)p_\theta(a|w)$. The auxiliary function $j(w,a)$ gives the objective for a single $(w,a)$ decision path and allows us to rewrite the objective to $J(\theta) = \sum_{w,a}p(w,a)j(w,a)$. We can transform the derivative of the objective function into a expected value by applying the log trick and noticing that for each parametrized function $f_\theta(x)$ it holds that $\sum_x (\nabla_\theta p_\theta(x))f_\theta(x) = \sum_x p_\theta(x)(\nabla_\theta \log f_\theta(x))$. Similar to Policy Gradient methods (Sutton et al., 2000), we arrive at the following objective function at its gradient:

\begin{eqnarray}
J(\theta) = \mathbb{E}_{p(w,a)}\left[p_\theta(a \vert w) j(w,a) \right] \\
\nabla_\theta J = \mathbb{E}_{p(w,a)}\left[\nabla_\theta \log p_\theta(a \vert w) j(w,a) \right]
\end{eqnarray}

which we can approximate with Monte Carlo batches due to the expectation operator.

##### Approximating the Prior Policy
The prior policy is given by the marginal of the posterior policies, i.e. $p(a) = \sum_w p(w) p(a|w)$. Its' computation requires summing over all states, which can be infeasible for large state spaces. Instead, we keep a exopential running average $\hat{p}(a)$ of the posterior policies $p(a|w)$ (Ortega et al. 2015):

$$\hat{p}_{t+1} (a)= \lambda\hat{p}_{t}(a) + (1.0 - \lambda)p(a|w),$$

where $\lambda$ is a decay parameter we set close to 1, e.g. 0.99. Note, that this is feasible only in the discrete action case, such as in classification. For continous actions, we keep a running average over the parameters that form the action distribution, e.g. the mean and variance of a gaussian distribution. Using this approximation, the auxillary function $j(w,a)$ becomes $\U(w,a) - \frac{1}{\beta} \log \frac{p_\theta(a\vert w)}{\hat{p}(a)}$.

# Basic Regularization: Weight Decay
Regularization techniques are designed to reduce the overfitting of a classifier (or regressor) during training. Overfitting means, simply put, to learn the input-output relation perfectly. The most common one is weight decay, whereby a penalty is added to the utility function. The penalty is proportional to the L2 norm of the weights. Let $\U(w,a)$ be the utility function we want to optimize, e.g. classification error. The regularized objective function is then

\begin{eqnarray} \U_{\mathrm{reg}}(w,a) = -\U(w,a) + \frac{\lambda}{2n} \sum_\Theta \theta^2,
\end{eqnarray}

where $\lambda$ is the regularization factor, and $\Theta$ is the set of weight of the model. In the case of categorical cross entropy, this becomes

\begin{eqnarray} \U_{\mathrm{reg}}(w,a) = -\U(w,a)\log(p(a|w)) + \frac{\lambda}{2n} \sum_\Theta \theta^2.
\tag{85}\end{eqnarray}

This formulazation forces the model to have weights close to zero, hence the term weight decay.

## Implementation
Imlementation is done in Keras (Chollet et al., 2013). To keep the notation consistent, we will refer to inputs as (world states) $w$ and predictions as (actions) $a$.

### Running Average Layers
Running Average Layers are implemented as a sub-class of Dense Layers with a variable for each input.

In [ ]:
from keras.layers import *
import keras.backend as K


class ExponentialMovingAverage(Layer):
    """Keeps an exponential average of the input variables.

    This is useful to mitigate overfitting
    (you could see it as a Bounded Rational Decision Maker where the running averages are the
    prior strategies.)

    As it is a regularization layer, it is only active at training time.

    # Arguments
        momentum: float, momentum of the epxonential decay
        initializers: string, how to initialize the variables

    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.

    # Output shape
        Same shape as input.
    """

    def __init__(self, units=1, momentum=0.99, initiliazier='ones', **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ExponentialMovingAverage, self).__init__(**kwargs)
        self.supports_masking = True
        self.momentum = momentum
        self.units = units
        self.axis = -1
        self.initializer = initiliazier
        self.values = None
        self.input_spec = InputSpec(min_ndim=2)

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[self.axis]

        self.values = self.add_weight(shape=(1, self.units),
                                      initializer=self.initializer,
                                      name='erm',
                                      trainable=False)

        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)
        reduction_axes = list(range(len(input_shape)))
        del reduction_axes[self.axis]

        def update_erm():
            normed_training, mean, variance = K.normalize_batch_in_training(x=inputs, beta=None, gamma=None,
                                                                            reduction_axes=reduction_axes)
            self.add_update([K.moving_average_update(self.values,
                                                     mean,
                                                     self.momentum)],
                            inputs=inputs)
            return self.values

        return K.in_train_phase(update_erm(), self.values, training=training)

    def get_config(self):
        config = {'momentum': self.momentum}
        base_config = super(ExponentialMovingAverage, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

## The Classifier
The classifier is a feed-forward neural network following the current state fo the art architectures. As we are dealing images, we use 2D convolutions with relu nonlinearities and tanh nonlinearities for the dense layers. The corresponding output class is found a softmax axtivation function. The weight are initialized according to the Xavier scheme (He et al., 2015). We use ADAM (Kingma and Ba, 2014) to optimize the parameters of the network. The loss function is a categorical cross-entropy with the DKL as a penalty:

$$-\sum_{w=1}^M\U(w,a)\log(p(a|w)) + \log(p(a|w)) \frac{\log(p(a|w))}{\hat{p}(a)},$$

where $\U(w,a)$ is binary indication function, that is 1 if the predicted class $a$ is correct for input $w$ and 0 else, and $M$ is the number of classes. In this example we look at the MNIST dataset (LeCun 1998), so $M = 10$.

In [ ]:
from keras.models import Model
from keras.optimizers import *
from ExponentialMovingAverageLayer import *
import tensorflow as tf
from keras import backend as K


def create_image_classifier(input_shape, num_classes, beta):
    def cat_cross_beta(beta):
        beta_var = K.variable(beta)

        def categorical_crossentropy(target, output, from_logits=False, b=beta_var):
            """Categorical crossentropy between an output tensor and a target tensor.

            # Arguments
                target: A tensor of the same shape as `output`.
                output: A tensor resulting from a softmax
                    (unless `from_logits` is True, in which
                    case `output` is expected to be the logits).
                from_logits: Boolean, whether `output` is the
                    result of a softmax, or is a tensor of logits.

            # Returns
                Output tensor.
            """
            # Note: tf.nn.softmax_cross_entropy_with_logits
            # expects logits, Keras expects probabilities.
            clipped_y = K.clip(output, K.epsilon(), 1)
            kl_loss = clipped_y * (K.log(clipped_y / K.clip(erm, K.epsilon(), 1)))
            kl_loss = (K.variable(1.0) / K.variable(beta)) * K.mean(kl_loss)

            if not from_logits:
                # scale preds so that the class probas of each sample sum to 1
                output /= tf.reduce_sum(output,
                                        len(output.get_shape()) - 1,
                                        True)
                # manual computation of crossentropy
                output = tf.clip_by_value(output, K.epsilon(), 1. - K.epsilon())
                return - tf.reduce_sum(target * tf.log(output),
                                       len(output.get_shape()) - 1) - kl_loss
            else:
                return tf.nn.softmax_cross_entropy_with_logits(labels=target,
                                                               logits=output) - kl_loss

        return categorical_crossentropy

    inlayer = Input(shape=input_shape)
    h = Conv2D(32, kernel_size=(3, 3),
               activation='relu', kernel_initializer='he_normal', use_bias=True)(inlayer)
    h = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', use_bias=True)(h)
    h = Flatten()(h)
    h = Dense(128, activation='tanh', kernel_initializer='he_normal', use_bias=True)(h)
    h = Dense(num_classes, activation='softmax', kernel_initializer='he_normal', use_bias=True)(h)
    erm = ExponentialMovingAverage(units=num_classes, name="conv_erm")(h)
    model = Model(inlayer, [h, erm])

    model.compile(loss=[cat_cross_beta(beta=beta), None],
                  optimizer=Adam(),
                  metrics=['acc'])
    return model


## Putting it all together
Running this code may take some time.

In [ ]:
'''Trains a simple convnet on the MNIST dataset.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from BoundedImageClassifier import *
from matplotlib import pyplot as plt
from matplotlib import rc

rc('text', usetex=True)

batch_size = 128
num_classes = 10
epochs = 5

beta = 125

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

idx = np.random.choice(len(x_train), 5000)
x_train = x_train[idx]
y_train = y_train[idx]
idx = np.random.choice(len(x_train), 1000)
y_test = y_test[idx]
x_test = x_test[idx]

model = create_image_classifier(input_shape=input_shape, beta=beta, num_classes=num_classes)

hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=2,
                 validation_data=(x_test, y_test))

plt.plot(hist.history['dense_2_acc'], label='Training Accuracy')
plt.plot(hist.history['val_dense_2_acc'], label='Validation Accuracy')
plt.title(r"Training and Validation Loss for \beta = %.3f" % beta)
plt.legend()
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score)
print('Test accuracy:', score)
plt.show()


Genewein, T., Leibfried, F., Grau-Moya, J., & Braun, D. A. (2015). Bounded rationality, abstraction, and hierarchical decision-making: An information-theoretic optimality principle. Frontiers in Robotics and AI, 2, 27.


Ortega, P. A., Braun, D. A., Dyer, J., Kim, K. E., & Tishby, N. (2015). Information-Theoretic Bounded Rationality. arXiv preprint arXiv:1512.06789.

Sutton, R. S., McAllester, D. A., Singh, S. P., & Mansour, Y. (2000). Policy gradient methods for reinforcement learning with function approximation. In Advances in neural information processing systems (pp. 1057-1063).

Ortega, P. A., & Braun, D. A. (2013). Thermodynamics as a theory of decision-making with information-processing costs. Proc. R. Soc. A, 469(2153), 20120683.

Chollet, F. (2015). Keras. https://keras.io

Kingma, D. P., & Ba, J. (2014). Adam: A method for stochastic optimization. arXiv preprint arXiv:1412.6980.

LeCun, Y. (1998). The MNIST database of handwritten digits. http://yann.lecun.com/exdb/mnist/.

He, K., Zhang, X., Ren, S., & Sun, J. (2015). Delving deep into rectifiers: Surpassing human-level performance on imagenet classification. In Proceedings of the IEEE international conference on computer vision (pp. 1026-1034).